In [1]:
import pandas as pd
import os

from pprint import pprint
from azure.ai.evaluation import evaluate
from azure.ai.evaluation import SimilarityEvaluator

In [2]:
from dotenv import load_dotenv
load_dotenv(".env")
model_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.environ.get("AZURE_OPENAI_API_KEY"),
    "azure_deployment":os.environ.get("AZURE_OPENAI_DEPLOYMENT"),
}

In [ ]:
import evalprompt.generateApiSpec as generateApiSpec
import pathlib
# Generate output from the test dataset, the jsonl file under this output will be used as the evaluation dataset for the prompt
query='Please be professional, and use below infomation to generate an OpenAPI specification documentation with YAML format:'
api_folder = pathlib.Path(__file__).parent.resolve() / "./apisrc"
apispec_dir = pathlib.Path(__file__).parent.resolve() / "./apispec"
output_dir = pathlib.Path(__file__).parent.resolve() / "./output"
output_dir.mkdir(exist_ok=True)
generateApiSpec.generateOutputData(query, api_folder, apispec_dir, output_dir)

In [ ]:
import random
import pathlib
randomNum = random.randint(1111, 9999)
similarity_evaluator = SimilarityEvaluator(model_config)
data_path=pathlib.Path(__file__).parent.resolve() / "./output/output.jsonl"
results = evaluate(
    evaluation_name="Eval-Run-" + str(randomNum),
    data=data_path,
    evaluators={
        "relevance": similarity_evaluator,
    },
    evaluator_config={
        "relevance": {
            "ground_truth":"${data.ground_truth}",
            "response":"${data.response}",
            "query":"${data.query}",
        }
    }
)

In [ ]:
pprint(results)

In [6]:
pd.DataFrame(results["rows"])

,inputs.response,inputs.query,inputs.context,inputs.ground_truth,outputs.relevance.similarity,outputs.relevance.gpt_similarity
0,```yaml\nopenapi: 3.0.0\ninfo:\n title: Flask...,"Please be professional, and use below infomati...","from flask import Flask, jsonify, request \nfr...",openapi: 3.0.0\ninfo:\n title: API Specificat...,3,3
1,Here is the OpenAPI specification documentatio...,"Please be professional, and use below infomati...",const express = require('express');\nconst app...,openapi: 3.0.0\ninfo:\n title: Pets API\n ve...,3,3
2,Certainly! Below is the OpenAPI Specification ...,"Please be professional, and use below infomati...","[ApiController]\n[Route(""[controller]"")]\npubl...",openapi: 3.0.0\ninfo:\n title: Weather Foreca...,3,3
